In [1]:
import pandas as pd
import numpy as np

import os, gc, time

In [2]:
bureau = pd.read_csv('DATA/bureau.csv/bureau.csv')

In [14]:
bb = pd.read_csv('DATA/bureau_balance.csv/bureau_balance.csv')

Create new features

In [8]:
## Credit duration and credit end date difference
# Time
bureau['CREDIT_DURATION'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_CREDIT']
bureau['ENDATE_DIFF'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']

## Ratio and difference of debt
# Amount credit
bureau['DEBT_CREDIT_DIFF'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT']
bureau['DEBT_PERC'] = bureau['AMT_CREDIT_SUM_DEBT'] / bureau['AMT_CREDIT_SUM']
bureau['DEBT_CREDIT_TO_ANNUITY_RATIO'] = bureau['AMT_CREDIT_SUM_DEBT'] / bureau['AMT_ANNUITY']

In [9]:
## One hot encoder

def one_hot_encoder(df, nan_as_category = True):

    original_columns = list(df.columns)
    
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']

    # One hot encoder
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)

    # New columns created by get_dummies
    categorical_columns = [col for col in df.columns if col not in original_columns]

    return df, categorical_columns

In [10]:
## One hot encoder bureau
bureau, categorical_columns = one_hot_encoder(bureau)
# categorical_columns

['CREDIT_ACTIVE_Active',
 'CREDIT_ACTIVE_Bad debt',
 'CREDIT_ACTIVE_Closed',
 'CREDIT_ACTIVE_Sold',
 'CREDIT_ACTIVE_nan',
 'CREDIT_CURRENCY_currency 1',
 'CREDIT_CURRENCY_currency 2',
 'CREDIT_CURRENCY_currency 3',
 'CREDIT_CURRENCY_currency 4',
 'CREDIT_CURRENCY_nan',
 'CREDIT_TYPE_Another type of loan',
 'CREDIT_TYPE_Car loan',
 'CREDIT_TYPE_Cash loan (non-earmarked)',
 'CREDIT_TYPE_Consumer credit',
 'CREDIT_TYPE_Credit card',
 'CREDIT_TYPE_Interbank credit',
 'CREDIT_TYPE_Loan for business development',
 'CREDIT_TYPE_Loan for purchase of shares (margin lending)',
 'CREDIT_TYPE_Loan for the purchase of equipment',
 'CREDIT_TYPE_Loan for working capital replenishment',
 'CREDIT_TYPE_Microloan',
 'CREDIT_TYPE_Mobile operator loan',
 'CREDIT_TYPE_Mortgage',
 'CREDIT_TYPE_Real estate loan',
 'CREDIT_TYPE_Unknown type of loan',
 'CREDIT_TYPE_nan']

Execute bureau balance

In [15]:
## One hot encoder bureau balance
# STATUS has X value mean unknown
bb, bb_cat = one_hot_encoder(bb, nan_as_category= False)
bb_cat

['STATUS_0',
 'STATUS_1',
 'STATUS_2',
 'STATUS_3',
 'STATUS_4',
 'STATUS_5',
 'STATUS_C',
 'STATUS_X']

In [16]:
## Agg mean for all categorical columns and merge with original dataframe
bb_cat_agg = bb.groupby('SK_ID_BUREAU')[bb_cat].mean().reset_index()


,SK_ID_BUREAU,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X
0,5001709,0.000000,0.000000,0.0,0.0,0.0,0.0,0.886598,0.113402
1,5001710,0.060241,0.000000,0.0,0.0,0.0,0.0,0.578313,0.361446
2,5001711,0.750000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.250000
3,5001712,0.526316,0.000000,0.0,0.0,0.0,0.0,0.473684,0.000000
4,5001713,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...
817390,6842884,0.187500,0.000000,0.0,0.0,0.0,0.0,0.416667,0.395833
817391,6842885,0.500000,0.000000,0.0,0.0,0.0,0.5,0.000000,0.000000
817392,6842886,0.242424,0.000000,0.0,0.0,0.0,0.0,0.757576,0.000000
817393,6842887,0.162162,0.000000,0.0,0.0,0.0,0.0,0.837838,0.000000


In [24]:
agg = {'MONTHS_BALANCE': ['mean', 'max', 'min', 'size']} # size includes nan, count does not
agg_df = bb.groupby('SK_ID_BUREAU').agg(agg)
agg_df.columns = pd.Index(['{}{}_{}'.format('', ele[0], ele[1].upper()) for ele in agg_df.columns.tolist() ])

bb_agg = bb_cat_agg.merge(agg_df, on= 'SK_ID_BUREAU', how= 'left')
bb_agg

,SK_ID_BUREAU,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X,MONTHS_BALANCE_MEAN,MONTHS_BALANCE_MAX,MONTHS_BALANCE_MIN,MONTHS_BALANCE_SIZE
0,5001709,0.000000,0.000000,0.0,0.0,0.0,0.0,0.886598,0.113402,-48.0,0,-96,97
1,5001710,0.060241,0.000000,0.0,0.0,0.0,0.0,0.578313,0.361446,-41.0,0,-82,83
2,5001711,0.750000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.250000,-1.5,0,-3,4
3,5001712,0.526316,0.000000,0.0,0.0,0.0,0.0,0.473684,0.000000,-9.0,0,-18,19
4,5001713,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,1.000000,-10.5,0,-21,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
817390,6842884,0.187500,0.000000,0.0,0.0,0.0,0.0,0.416667,0.395833,-23.5,0,-47,48
817391,6842885,0.500000,0.000000,0.0,0.0,0.0,0.5,0.000000,0.000000,-11.5,0,-23,24
817392,6842886,0.242424,0.000000,0.0,0.0,0.0,0.0,0.757576,0.000000,-16.0,0,-32,33
817393,6842887,0.162162,0.000000,0.0,0.0,0.0,0.0,0.837838,0.000000,-18.0,0,-36,37


Merge bureau balance with bureau after one hot encoder

In [27]:
bureau = bureau.merge(bb_agg, on= 'SK_ID_BUREAU', how= 'left')

In [30]:
bureau['STATUS_SUM'] = 0
for i in range(1,6):
    bureau['STATUS_SUM'] += bureau['STATUS_{}'.format(i)]

Aggregate bureau

In [73]:
agg_bureau = bureau.groupby('SK_ID_CURR').agg(BUREAU_AGG)
agg_bureau.columns = pd.Index(['{}{}_{}'.format('BUREAU_', ele[0], ele[1].upper()) for ele in agg_bureau.columns.tolist()])

In [72]:
BUREAU_AGG = {
    'SK_ID_BUREAU' : ['nunique'],
    'DAYS_CREDIT': ['mean', 'max', 'min'],
    'DAYS_CREDIT_ENDDATE': ['mean'],
    'CREDIT_DAY_OVERDUE': ['mean', 'max'],
    'AMT_CREDIT_MAX_OVERDUE': ['mean', 'max'],
    'AMT_CREDIT_SUM': ['mean', 'max', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'max', 'sum'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean', 'max'],
    'AMT_ANNUITY': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean', 'sum'],
    'MONTHS_BALANCE_MEAN': ['mean', 'var'],
    'MONTHS_BALANCE_SIZE': ['mean', 'sum'],
    # Categorical 
    'STATUS_0': ['mean'],
    'STATUS_SUM': ['mean'],
    'STATUS_C': ['mean'],
    'STATUS_X': ['mean'],
    'CREDIT_ACTIVE_Active': ['mean'],
    'CREDIT_ACTIVE_Closed' :['mean'],
    'CREDIT_ACTIVE_Sold' :['mean'],
    'CREDIT_ACTIVE_Bad debt': ['mean'],
    
    'CREDIT_TYPE_Car loan': ['mean'],
    'CREDIT_TYPE_Cash loan (non-earmarked)': ['mean'],
    'CREDIT_TYPE_Consumer credit': ['mean'],
    'CREDIT_TYPE_Credit card': ['mean'],
    'CREDIT_TYPE_Interbank credit': ['mean'],
    'CREDIT_TYPE_Loan for business development': ['mean'],
    'CREDIT_TYPE_Loan for purchase of shares (margin lending)': ['mean'],
    'CREDIT_TYPE_Loan for the purchase of equipment': ['mean'],
    'CREDIT_TYPE_Loan for working capital replenishment': ['mean'],
    'CREDIT_TYPE_Microloan': ['mean'],
    'CREDIT_TYPE_Mobile operator loan': ['mean'],
    'CREDIT_TYPE_Mortgage': ['mean'],
    'CREDIT_TYPE_Real estate loan': ['mean'],
    'CREDIT_TYPE_Unknown type of loan': ['mean']

}

Active and Closed loans aggregates

In [75]:
active = bureau.loc[bureau['CREDIT_ACTIVE_Active'] == 1]
agg_bureau = group_and_merge(active, agg_bureau, 'BUREAU_ACTIVE_', BUREAU_ACTIVE_CLOSED_AGG)

In [74]:
BUREAU_ACTIVE_CLOSED_AGG = {
    'DAYS_CREDIT': ['mean', 'max', 'min'],
    'CREDIT_DAY_OVERDUE': ['mean', 'max'],
    'DAYS_CREDIT_ENDDATE': ['mean'],
    'AMT_CREDIT_MAX_OVERDUE': ['mean', 'max'],
    'AMT_CREDIT_SUM': ['mean', 'max', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'max', 'sum'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean', 'max'],
    'AMT_ANNUITY': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean', 'sum'],
    'DEBT_PERC': ['mean'],
    'STATUS_SUM': ['mean'],
    'DEBT_CREDIT_TO_ANNUITY_RATIO': ['mean'],
    'MONTHS_BALANCE_MEAN': ['mean', 'var'],
    'MONTHS_BALANCE_SIZE': ['mean', 'sum']
}

In [76]:
closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
agg_bureau = group_and_merge(closed, agg_bureau, 'BUREAU_CLOSED_', BUREAU_ACTIVE_CLOSED_AGG)

In [77]:
## Ratios: total debt/ total credit and active loan debt/ active loan credit
agg_bureau['BUREAU_DEBT_TO_CREDIT_RATIO'] = agg_bureau['BUREAU_AMT_CREDIT_SUM_DEBT_SUM'] / agg_bureau['BUREAU_AMT_CREDIT_SUM_SUM']
agg_bureau['BUREAU_ACTIVE_DEBT_TO_CREDIT_RATIO'] = agg_bureau['BUREAU_ACTIVE_AMT_CREDIT_SUM_DEBT_SUM'] / agg_bureau['BUREAU_ACTIVE_AMT_CREDIT_SUM_SUM']

group time to aggregate

In [84]:
for months in [6,12,18,24]:
    prefix = "BUREAU_LAST_{}_M_".format(months)
    months_df = bureau[bureau['DAYS_CREDIT'] >= -30 * months]
    agg_bureau = group_and_merge(months_df, agg_bureau, prefix, BUREAU_TIME_AGG)

In [83]:
BUREAU_TIME_AGG = {
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM': ['max', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'sum'],
    'DEBT_PERC': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean'],
    'STATUS_0': ['mean'],
    'STATUS_SUM': ['mean'],
}

In [85]:
agg_bureau.to_parquet('TO_TRAIN/bureau.gzip', compression= 'gzip', index= False)

In [87]:
bureau_T = pd.read_parquet('TO_TRAIN/bureau.gzip')
bureau_T

,SK_ID_CURR,BUREAU_SK_ID_BUREAU_NUNIQUE,BUREAU_DAYS_CREDIT_MEAN,BUREAU_DAYS_CREDIT_MAX,BUREAU_DAYS_CREDIT_MIN,BUREAU_DAYS_CREDIT_ENDDATE_MEAN,BUREAU_CREDIT_DAY_OVERDUE_MEAN,BUREAU_CREDIT_DAY_OVERDUE_MAX,BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN,BUREAU_AMT_CREDIT_MAX_OVERDUE_MAX,...,BUREAU_LAST_24_M_AMT_CREDIT_MAX_OVERDUE_MEAN,BUREAU_LAST_24_M_AMT_CREDIT_SUM_OVERDUE_MEAN,BUREAU_LAST_24_M_AMT_CREDIT_SUM_MAX,BUREAU_LAST_24_M_AMT_CREDIT_SUM_SUM,BUREAU_LAST_24_M_AMT_CREDIT_SUM_DEBT_MEAN,BUREAU_LAST_24_M_AMT_CREDIT_SUM_DEBT_SUM,BUREAU_LAST_24_M_DEBT_PERC_MEAN,BUREAU_LAST_24_M_DEBT_CREDIT_DIFF_MEAN,BUREAU_LAST_24_M_STATUS_0_MEAN,BUREAU_LAST_24_M_STATUS_SUM_MEAN
0,100001,7,-735.000000,-49,-1572,82.428571,0.0,0,NaN,NaN,...,NaN,0.0,378000.0,884025.000,1.988955e+05,596686.50,0.659208,95779.5000,0.710526,0.017544
1,100002,8,-874.000000,-103,-1437,-349.000000,0.0,0,1681.029,5043.645,...,2542.0725,0.0,120735.0,152723.565,0.000000e+00,0.00,0.000000,50907.8550,0.395833,0.166667
2,100003,4,-1400.750000,-606,-2586,-544.500000,0.0,0,0.000,0.000,...,0.0000,0.0,810000.0,810000.000,0.000000e+00,0.00,0.000000,810000.0000,NaN,NaN
3,100004,2,-867.000000,-408,-1326,-488.500000,0.0,0,0.000,0.000,...,0.0000,0.0,94537.8,94537.800,0.000000e+00,0.00,0.000000,94537.8000,NaN,NaN
4,100005,3,-190.666667,-62,-373,439.333333,0.0,0,0.000,0.000,...,0.0000,0.0,568800.0,657126.000,1.894695e+05,568408.50,0.601256,29572.5000,0.735043,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305806,456249,13,-1667.076923,-483,-2713,-1232.333333,0.0,0,6147.000,18945.000,...,NaN,0.0,180000.0,180000.000,1.630710e+05,163071.00,0.905950,16929.0000,NaN,NaN
305807,456250,3,-862.000000,-760,-1002,1288.333333,0.0,0,0.000,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305808,456253,4,-867.500000,-713,-919,280.500000,0.0,0,NaN,NaN,...,NaN,0.0,2250000.0,2250000.000,1.624797e+06,1624797.00,0.722132,625203.0000,0.458333,0.000000
305809,456254,1,-1104.000000,-1104,-1104,-859.000000,0.0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Function to aggregate numeric columns 
def group(df_to_agg, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = df_to_agg.groupby(aggregate_by).agg(aggregations)
    agg_df.columns = pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                               for e in agg_df.columns.tolist()])
    return agg_df.reset_index()

# Function to merge numeric and categorical after aggregate
def group_and_merge(df_to_agg, df_to_merge, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = group(df_to_agg, prefix, aggregations, aggregate_by= aggregate_by)
    return df_to_merge.merge(agg_df, how='left', on= aggregate_by)